<a href="https://colab.research.google.com/github/pacifist2301/ChatWall/blob/master/EyeBlinkModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip researchdataset1.zip

In [ ]:
import tensorflow as tf
train_dir ='train'
test_dir  ='test'

In [ ]:
width, height = 86, 86
training=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
                                                          rotation_range=7,
                                                          horizontal_flip=True,
                                                          validation_split=0.05
                                                         ).flow_from_directory(train_dir,
                                                                               class_mode = 'categorical',
                                                                               batch_size = 8,
                                                           target_size=(width,height),
                                                                              subset="training")
testing=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
                                                         ).flow_from_directory(test_dir,
                                                                               class_mode = 'categorical',
                                                                               batch_size = 8,
                                                                               shuffle = False,
                                                           target_size=(width,height))
validing=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
                                                          rotation_range=7,
                                                          horizontal_flip=True,
                                                         validation_split=0.05
                                                        ).flow_from_directory(train_dir,
                                                                              batch_size = 8,
                                                                              class_mode = 'categorical',
                                                           target_size=(width,height),subset='validation',shuffle=True)

In [ ]:
from keras.models import Sequential ,Model
from keras.layers import Dense ,Flatten ,Conv2D ,MaxPooling2D ,Dropout ,BatchNormalization  ,Activation ,GlobalMaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping ,ReduceLROnPlateau

In [ ]:
optimizer=Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.99)
EarlyStop=EarlyStopping(monitor='val_accuracy',patience=10,restore_best_weights=False)
#Reduce_LR=ReduceLROnPlateau(monitor='val_accuracy',verbose=2,factor=0.5,min_lr=0.00001)
callback=[EarlyStop]

In [ ]:
num_classes = 2
#num_detectors=32

network = Sequential()

network.add(Conv2D(128,(3,3), activation='relu', padding = 'same', input_shape = (86, 86, 3)))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2,2)))
network.add(Dropout(0.4))

network.add(Conv2D(128,(3,3), activation='relu', padding = 'same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2,2)))
network.add(Dropout(0.3))

network.add(Flatten())

network.add(Dense(64, activation='relu'))
network.add(Dropout(0.2))

network.add(Dense(2, activation='softmax'))

In [ ]:
network.summary()

In [ ]:
network.compile(optimizer='adam',loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
history=network.fit(training,validation_data=validing,epochs=10, callbacks=callback, verbose='auto')

In [ ]:
val,los=network.evaluate(testing)

In [ ]:
import matplotlib.pyplot as plt

metrics = history.history
plt.plot(history.epoch, metrics['loss'])
plt.legend(['loss'])
plt.show()

In [ ]:
network.save('finaleyemodel.h5')

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model('finaleyemodel.h5')

In [ ]:
filenames = testing.filenames
true_labels = testing.classes

In [ ]:
predicted_labels = []
predicted_probabilities = []

In [ ]:
import os
import numpy as np
for i in range(len(filenames)):
    # Load and preprocess the image
    img_path = os.path.join(test_dir, filenames[i])
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(width, height))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize the image

    # Predict the class label
    predictions = model.predict(img_array)
    predicted_label = np.argmax(predictions)

    # Append the predicted label and probabilities
    predicted_labels.append(predicted_label)
    predicted_probabilities.append(predictions)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
predicted_labels = np.array(predicted_labels)
predicted_probabilities = np.array(predicted_probabilities)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f'Accuracy: {accuracy:.2f}')

# Generate a classification report
report = classification_report(true_labels, predicted_labels)
print(f'Classification Report:\n{report}')

# Create a confusion matrix
confusion = confusion_matrix(true_labels, predicted_labels)
print(f'Confusion Matrix:\n{confusion}')

# Display images with true and predicted labels
for i in range(len(filenames)):
    img_path = os.path.join(test_dir, filenames[i])
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(width, height))

    true_class_name = 'closed-eye' if true_labels[i] == 0 else 'open-eye'
    predicted_class_name = 'closed-eye' if predicted_labels[i] == 0 else 'open-eye'

    plt.figure()
    plt.imshow(img)
    plt.title(f'True: {true_class_name}, Predicted: {predicted_class_name}')
    plt.axis('off')
    plt.show()